In [ ]:
1+1

In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau
from multiprocessing import Pool, cpu_count

import importlib
import os
from tqdm.auto import tqdm

# local imports
from lib.constants import *

from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
dosave = True
plotqual = ""
tqual = ""
savedir = "plots_6_7_25/%s" % plotqual
plt.rcParams.update({'font.size': 14})
plottitle = "Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]


filedirs = [
    "/exp/icarus/data/users/gputnam/ICARUS-calibration-2D/validation/FNAL/",
]

outdir = "/exp/icarus/data/users/gputnam/ICARUS-calibration-2D/validation/FNAL/"

savedata = True
PLANE = 2
outf = outdir + "P%i_scaleYZ_Run2_2D.txt" % PLANE

isMC = False

In [ ]:
files = sum([[filedir + f for f in os.listdir(filedir) if f.endswith(".df") and 
              "yz" in f and ("P%i" % PLANE) in f and "C1" not in f] for filedir in filedirs], [])
files

In [ ]:
import gc
gc.collect()

In [ ]:
NCHUNK = 10

In [ ]:
yz_ybin = np.linspace(-180, 130, 32)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 181)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.


yz_ybin = np.linspace(-180, 130, 16)
yz_ylos = yz_ybin[:-1]
yz_yhis = yz_ybin[1:]
yz_ys = (yz_ylos + yz_yhis) / 2.

yz_zbin = np.linspace(-900, 900, 91)
yz_zlos = yz_zbin[:-1]
yz_zhis = yz_zbin[1:]
yz_zs = (yz_zlos + yz_zhis) / 2.
    

wbin = np.linspace(0, 1701, int(1701 / 0.3) + 1)
wlo = wbin[:-1]
whi = wbin[1:]
ws = (wlo + whi) / 2.

itpc_bins = np.linspace(0, 4, 5) - 0.5

tpcs = ["EE", "EW", "WE", "WW"]


qbins = np.linspace(100, 4000, 39*4+1)
qbin_centers = (qbins[1:] + qbins[:-1]) / 2. 

In [ ]:
lifetime_file = outdir + "lifetimes_runArun1run2.txt"
bootstrap = False
scaleyz_file = outdir + "P2_scaleYZ_RunA.txt"

DQDX_NAME = "dqdx_normt"

In [ ]:
def process_file(inp):
    i, f = inp
    
    # print("loading!")
    df = pd.read_hdf(f)
    for c in df.columns:
        if c.startswith("true") or c.startswith("dir"):
            del df[c]
    del df["sumadc"]
    del df["width"]
    
    # print("loaded!")
    
    df["tpcEE"] = df.tpcE & (df.cryostat == 0)
    df["tpcEW"] = df.tpcW & (df.cryostat == 0)
    df["tpcWE"] = df.tpcE & (df.cryostat == 1)
    df["tpcWW"] = df.tpcW & (df.cryostat == 1)

    df["dqdx"] = df.charge / df.pitch
    df["thitp"] = (df.time * tick_period - df.pandora_t0 - tanode*tick_period) / 1000.

    if "bnb" in f:
        df["bnb"] = True
    else:
        df["bnb"] = False
        
    df["ifile"] = i

    df["lifetime"] = np.nan

    fileruns = df.run.unique()
    if not isMC:
        with open(lifetime_file) as f:
            next(f)
            for line in f:
                dat = list(map(float, line.rstrip("\n").split(" ")))
                run = int(dat[0])
                
                if run not in fileruns: continue
                
                df.loc[(df.run == run) & df.tpcEE, "lifetime"] = dat[1]
                df.loc[(df.run == run) & df.tpcEW, "lifetime"] = dat[2]
                df.loc[(df.run == run) & df.tpcWE, "lifetime"] = dat[3]
                df.loc[(df.run == run) & df.tpcWW, "lifetime"] = dat[4]
    else:
        df.lifetime = 3.5e3
        
    # print("Made lifetimes!")

    df["itpc"] = -1
    df.loc[df.tpcEE, "itpc"] = 0
    df.loc[df.tpcEW, "itpc"] = 1
    df.loc[df.tpcWE, "itpc"] = 2
    df.loc[df.tpcWW, "itpc"] = 3

    df["scale_yz"] = np.nan

    if bootstrap and not isMC:

        df["ybin"] = np.searchsorted(yz_ybin, df.y.values) - 1
        df["zbin"] = np.searchsorted(yz_zbin, df.z.values) - 1

        idx = []
        scales = []

        with open(scaleyz_file) as f:
            next(f)
            for line in f:
                dat = line.rstrip("\n").split("\t")
                tpc = dat[0]
                iy = int(dat[1])
                iz = int(dat[2])
                scale = float(dat[3])

                idx.append((tpcnames.index(tpc), iy, iz))
                scales.append(scale)

        scaledf = pd.DataFrame(scales, index=pd.MultiIndex.from_tuples(idx, names=["itpc", "ybin", "zbin"]), 
                               columns=["scale_yz"])
        dtmp = pd.merge(df[["itpc", "ybin", "zbin"]], scaledf, on=["itpc", "ybin", "zbin"], how="left")

        df["scale_yz"] = dtmp.scale_yz.values

        del dtmp

    else:
        df.scale_yz = 1

    df["dqdx_normt"] = df.dqdx * np.exp(df.thitp / df.lifetime) / df.scale_yz
    
    when = (df.pitch > 0.3*NCHUNK) & (df.pitch < 0.5*NCHUNK) & (df.thitp > 50) & (df.thitp < 900) #&\
    Ndqdx, _ = np.histogramdd((df.itpc[when], df.y[when], df.z[when], df[DQDX_NAME][when]),
                         bins=[itpc_bins, yz_ybin, yz_zbin, qbins])
    
    wCE = -(df.y - 134.95)*np.sin(30*np.pi/180) + (df.z + 894.950652270838)*np.cos(30*np.pi/180)
    wCW = (df.y + 181.85)*np.sin(30*np.pi/180) + (df.z + 894.950652270838)*np.cos(30*np.pi/180)
    
    df["wC"] = wCE
    df.loc[~df.tpcE, "wC"] = wCW[~df.tpcE]

    df["wI2"] = wCW
    df.loc[~df.tpcE, "wI2"] = wCE[~df.tpcE]
    
    # print("Histogramming!")
    
    Ndqdx_wC, _ = np.histogramdd((df.itpc[when], df.wC[when], df.dqdx_normt[when]),
                     bins=[itpc_bins, wbin, qbins])
    
    Ndqdx_wI2, _ = np.histogramdd((df.itpc[when], df.wI2[when], df.dqdx_normt[when]),
                     bins=[itpc_bins, wbin, qbins])
    
    return Ndqdx, Ndqdx_wC, Ndqdx_wI2

In [ ]:
Nyzs = []
NCs = []
NIs = []

with Pool(processes=12) as pool:
    for Nyz, NwC, NwI in tqdm(pool.imap_unordered(process_file, enumerate(files)), total=len(files)):
        Nyzs.append(Nyz)
        NCs.append(NwC)
        NIs.append(NwI)

# for Nyz, NwC, NwI in tqdm(map(process_file, enumerate(files)), total=len(files)):
#     Nyzs.append(Nyz)
#     NCs.append(NwC)
#     NIs.append(NwI)
    
    

In [ ]:
qbins_centers = (qbins[:1] + qbins[:-1]) /2

In [ ]:
Ndqdx = np.sum(Nyzs, axis=0)

Ndqdx_wC = np.sum(NCs, axis=0)

Ndqdx_wI2 = np.sum(NIs, axis=0)

In [ ]:
def landau_gaus(X, *p):
    mpv, eta, sigma, A = p
    sigma = np.minimum(sigma, 100*eta)
    return landau.landau.gauss_landau(X, mpv, eta, sigma, A)


In [ ]:
def get_landau_data(N, fig=None, text="", title="", saveplt=""):
    maxbin = np.argmax(N)
    when_fit = np.abs(np.array(range(len(qbin_centers))) - maxbin) < 15 # within 15 bins

    mpv_lo = 100
    mpv_hi = 1000
    mpv0 = 750
    p0 = [mpv0, 50, 50, np.max(N)*400.]
    bounds = ([mpv_lo, 10, 10, np.max(N)*400./100.], [mpv_hi, 250, 250, np.max(N)*400.*100.])
    if np.max(N) == 0:
        return None

    popt, perr = curve_fit(landau_gaus, qbin_centers[when_fit], N[when_fit], 
                           p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

    MPV = popt[0] - 0.228*popt[1]
    MPV_err = np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0]))

    loc = popt[0]
    loc_err = np.sqrt(np.diag(perr)[0])
    eta = popt[1]
    sigma = popt[2]

    if fig:
        fig.hist(qbin_centers, weights=N, bins=qbins, label="Data")
        fig.plot(qbin_centers, landau_gaus(qbin_centers, *popt), label="Landau+Gaus Fit")
        fig.xlabel("dQ/dx [ADDC/cm]")
        fig.ylabel("# Depositions")
        fig.title(title)
        fig.xlim([200, 2500])
        fig.legend()
        fig.tight_layout()
        if saveplt and dosave:
            fig.savefig(saveplt)



    return MPV, MPV_err, loc, loc_err, eta, sigma

In [ ]:
def inner(inp):
    (tpc, zlo, zhi, ylo, yhi, iz, iy) = inp
    thisN = Ndqdx[tpcs.index(tpc), iy, iz, :]
    count = thisN.sum()
    fig = plt if PLOT else None
    dat = get_landau_data(thisN, fig, 
                        text=["%.0f < z < %.0fcm" % (zlo, zhi), "TPC EW"],
                        saveplt=savedir + "dQdx_z_%i_%i_TPCEW.png" % (int(zlo), int(zhi)))
    if dat is None:
        m = np.nan
        merr = np.nan
    else:
        m, merr, _, _, _, _ = dat
        
    return iz, iy, m, merr, count

PLOT = True

In [ ]:
inner(("EE", 0, 50, 0, 50, 50, 7))
PLOT = False

In [ ]:
zy_mpvs = {}
zy_mpverrs = {}
zy_Ns = {}

for tpc in tpcs:
    zy_mpvs[tpc] = []
    zy_mpverrs[tpc] = []
    zy_Ns[tpc] = []
    for _ in yz_ys:
        zy_mpvs[tpc].append([])
        zy_mpverrs[tpc].append([])
        zy_Ns[tpc].append([])
        
        for _ in yz_zs:
            zy_mpvs[tpc][-1].append(-1)
            zy_mpverrs[tpc][-1].append(-1)
            zy_Ns[tpc][-1].append(-1)
        

with Pool(processes=24) as pool:
    with tqdm(total=len(tpcs)*len(yz_zlos)*len(yz_ylos)) as pbar:
        for tpc in tpcs:
            for iy, (ylo, yhi) in enumerate(zip(yz_ylos, yz_yhis)):
                inp = [(tpc, zlo, zhi, ylo, yhi, iz, iy) for iz, (zlo, zhi) in enumerate(zip(yz_zlos, yz_zhis))]
                for row in pool.imap_unordered(inner, inp):
                    iz, _, m, merr, N = row

                    zy_mpvs[tpc][iy][iz] = m
                    zy_mpverrs[tpc][iy][iz] = merr
                    zy_Ns[tpc][iy][iz] = N

                pbar.update(len(yz_zlos))

In [ ]:
mean_scale = {}
for tpc in tpcs:
    val_arr = np.array(zy_mpvs[tpc])
    err_arr = np.array(zy_mpverrs[tpc])
    mean_scale[tpc] = np.mean(val_arr[~np.isnan(val_arr) & (err_arr / val_arr < 0.1)])

In [ ]:
if savedata:
    with open(outf, "w") as f:
        f.write("\t".join(["TPC", "Y-bin", "Z-bin", "Scale", "MPV", "MPV-Err", "N", "Y-lo", "Y-hi", "Z-lo", "Z-hi"]) + "\n")
        for tpc in tpcs:
            for iy in range(len(yz_ylos)):
                for iz in range(len(yz_zlos)):
                    f.write("\t".join(map(str, [tpc, iy, iz, zy_mpvs[tpc][iy][iz] / mean_scale[tpc], zy_mpvs[tpc][iy][iz], zy_mpverrs[tpc][iy][iz], zy_Ns[tpc][iy][iz],yz_ylos[iy], yz_yhis[iy], yz_zlos[iz], yz_zhis[iz]])) + "\n")

In [ ]:
outf